# Simulate depletion

This notebook process the CoNDERC data that contains both experimental and simulation.

Running this notebook also performs OpenMC depletion simulations for every experiment.

This can take over an hour on a typical laptop but is needed for production of all the results.

In [ ]:
import copy
import shutil

from pathlib import Path
from zipfile import ZipFile
from urllib.request import urlopen, Request

import numpy as np
import matplotlib.pyplot as plt
import openmc
import pypact as pp  # needs latest pypact which can be installed with pip install git+https://github.com/fispact/pypact

from openmc_activator import OpenmcActivator

This downloads and extracts the CoNDERC data. This contains the FNS experimental data and FISPACT inputs and outputs.

In [2]:
# download zip file
conderc_url = 'https://nds.iaea.org/conderc/fusion/files/fns.zip'
p = Request(conderc_url, headers={'User-Agent': 'Mozilla/5.0'})
with urlopen(p) as response, open('fns.zip', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

# unzip
with ZipFile('fns.zip', 'r') as f:
    f.extractall('.')

Read all the experiments from unzipped fns folder.

In [3]:
here = Path('./fns')
assert(here.exists()), 'fns folder does not seem to exist. Run `download_fns_fusion_decay.py` first to download and unzip FNS benchmark files.'
experiments = {}
files = [q for q in here.glob('*') if q.is_dir()]
for f in files:
    if '_' in f.name: continue
    l = list(f.glob('*fluxes*'))
    experiments[f.name] = []
    for name in l:
        x = name.name.replace('_fluxes', '')
        experiments[f.name].append(x)

Reads the Fispact fluxes file that contains the neutron spectra

In [4]:
# read flux data
flux_dict = {}
for k,l in experiments.items():
    flux_dict[k] = {}
    for exp in l:
        ff = pp.FluxesFile()
        pp.from_file(ff, here / k / (exp+'_fluxes'))
        assert(len(ff.values) == 709)
        ebins = ff.boundaries
        flux_dict[k][exp] = ff.values

Plot and example irradiation neutron spectra.

This example plots the neutron spectra used to irradiate silver (Ag) in the 2000 experimental campaign for 5 minutes of irradiation.

In [5]:
# in this case we plot the silver Ag experiment spectrum but you could plot others
plt.stairs(values=flux_dict['Ag']['2000exp_5min'], edges=ebins)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Energy [eV]')  #TODO check these units
plt.ylabel('Flux [n/cm$^2$/s]')  #TODO check these units
plt.close()

Next we read in the experimental data so that it is in a more accessible form.
The times, data and uncertainties are read in.

In [6]:
def read_experimental_data(exp_file):
    lines = open(exp_file).readlines()
    minutes = [q.strip().split()[0] for q in lines]
    # it's not always days actually, check other file
    vals = [q.strip().split()[1] for q in lines]
    unc = [q.strip().split()[2] for q in lines]
    return np.array(minutes, dtype=float), np.array(vals, dtype=float), np.array(unc, dtype=float)

# TODO consider replacing with pypact
def is_days(input_file):
    lines = open(input_file, 'r').readlines()
    lines = [q for q in lines if 'TIME' in lines]
    day_cnt = 0
    for line in lines:
        if 'DAYS' in line: day_cnt += 1
    if day_cnt == len(lines):
        return True
    elif day_cnt == 0:
        return False
    else:
        raise ValueError('Something is not right')

exp_data_dict = {'minutes': {}, 'data': {}, 'unc': {}}
for k,l in experiments.items():
    for k_ in exp_data_dict:
        exp_data_dict[k_][k] = {}
    for exp in l:
        exp_path = here / k / (exp+'.exp')
        exp_path = str(exp_path.absolute())
        input_path = here / k / ('TENDL-2017_' + exp + '.i')
        input_path = str(input_path.absolute())
        is_day = is_days(input_path)
        mins, vals, uncs = read_experimental_data(exp_path)
        if is_day:
            exp_data_dict['minutes'][k][exp] = mins * 60 * 24
        else:
            exp_data_dict['minutes'][k][exp] = mins
        exp_data_dict['data'][k][exp] = vals
        exp_data_dict['unc'][k][exp] = uncs
        assert(len(mins) == len(vals))

Now we get the irradiation setup including the flux and timesteps

In [ ]:
def read_irr_setup(filepath):
    ff = pp.InputData()
    pp.from_file(ff, filepath)
    cleaned_irradschedule = [item for item in ff._irradschedule if item != (0.0, 0.0)]
    flux_mag_list = [val[1] for val in cleaned_irradschedule] + [0.0] * len(ff._coolingschedule)
    days_list = np.cumsum([val[0] for val in cleaned_irradschedule] + ff._coolingschedule)/ (24*60*60)
    return days_list, flux_mag_list

def read_mat_setup(filepath):
    ff = pp.InputData()
    pp.from_file(ff, filepath)
    return ff._inventorymass.entries

def read_density(filepath):
    ff = pp.InputData()
    pp.from_file(ff, filepath)
    return ff._density

setup_dict = {'days': {}, 'flux_mag': {}, 'mass': {}, 'density': {}}
for k,l in experiments.items():
    for k_ in setup_dict:
        setup_dict[k_][k] = {}
    for exp in l:
        input_path = here / k / ('TENDL-2017_' + exp + '.i')
        input_path = str(input_path.absolute())
        days, flux_mag = read_irr_setup(input_path)
        mass_dict = {k:v/100 for k,v in read_mat_setup(input_path)}
        setup_dict['days'][k][exp] = days
        setup_dict['flux_mag'][k][exp] = flux_mag
        setup_dict['mass'][k][exp] = mass_dict
        setup_dict['density'][k][exp] = read_density(input_path)
        assert(len(days) == len(flux_mag))
        assert(isinstance(mass_dict, dict))

setup_dict['mg_flux'] = flux_dict
setup_dict['ebins'] = ebins

Now we can carry out depletion simulations in OpenMC

Set the chain file and cross sections to let OpenMC know where to find the data.

The nuclear data used can have an impact on how closely the results match.

To make this a fair comparison we recommend using the same nuclear data as the original Fispact simulations (Tendl 2017) and the chain file provided within the repository.

In [8]:
# Setting the cross section path to the location used by the CI.
# If you are running this locally you will have to change this path to your local cross section path.
openmc.config['cross_sections'] = Path.home() / 'nuclear_data' / 'cross_sections.xml'

# Setting the chain file to the relative path of the chain file included in the repository.
# Also resolving the chain file to the absolute path which is needed till the next release of OpenMC.
openmc.config['chain_file'] = Path('./fns_spectrum.chain.xml').resolve()

Next we use the experiment descriptions to make OpenMC simulations

The irradiation duration, spectra, flux, material and mass are found from the IAEA Conderc benchmarks and passed to OpenMC functions to perform simulations of the experimental setup.

In [ ]:
openmc_result_dict = {}
all_activation_data = []
element_exp_names = []
for k, l in experiments.items():
    
    # this loop currently just simulates the all materials in the benchmark suite
    # it can be changed to simulate a single material by commenting the line below.
    # if k != 'Ag': continue
    # or it it can be changed to simulate a two materials by commenting the line below.
    # if k not in ['Cd', 'NiCr']: continue

    print(f'Running OpenMC for {k} {l}')
    
    if k not in openmc_result_dict:
        openmc_result_dict[k] = {}
    for exp in l:
        if exp in openmc_result_dict[k]:
            continue
        ccfe_flux = flux_dict[k][exp]
        # ebins is ccfs 709 flux bins
        # low to high
        # create new chain file

        # mass in grams
        mass_dict = setup_dict['mass'][k][exp]
        days_list = setup_dict['days'][k][exp]
        # days are cumulative, so we gotta provide diffs
        days_list = np.append(days_list[0], np.diff(days_list))
        flux_mag_list = setup_dict['flux_mag'][k][exp]

        # make openmc material
        mat = openmc.Material()
        for el, md in mass_dict.items():
            el = el.lower().capitalize()
            mat.add_element(el, md, percent_type='wo')
        mat.set_density('g/cm3', setup_dict['density'][k][exp])
        mat.depletable = True
        mat.temperature = 294
        tot_mass = sum(mass_dict.values())
        mat.volume = tot_mass / mat.density

        activation_data = {
            'materials': mat,
            'multigroup_flux': ccfe_flux,
            'energy': ebins,
            'source_rate': flux_mag_list,
            'timesteps': days_list
        }
        element_exp_names.append((k,exp))
        all_activation_data.append(activation_data)

Running OpenMC for Tl ['2000exp_5min']
Running OpenMC for Rh ['2000exp_5min']
Running OpenMC for Na ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Pb ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for V ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Sr ['2000exp_5min', '1996exp_7hour', '1996exp_5min']


Running OpenMC for Ta ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Eu ['2000exp_5min']
Running OpenMC for Sn ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for As ['2000exp_5min']
Running OpenMC for Cs ['2000exp_5min']
Running OpenMC for Gd ['2000exp_5min']
Running OpenMC for F ['2000exp_5min', '1996exp_5min']
Running OpenMC for W ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Mg ['2000exp_5min']
Running OpenMC for Rb ['2000exp_5min']
Running OpenMC for Ge ['2000exp_5min']
Running OpenMC for S ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Zn ['2000exp_5min']
Running OpenMC for Ho ['2000exp_5min']
Running OpenMC for Pr ['2000exp_5min']
Running OpenMC for Bi ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Sb ['2000exp_5min']
Running OpenMC for Ni ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Sc ['2000exp_5min']
Running OpenMC for Ag ['2000exp_5min']
Running O

Running OpenMC for Ba ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Mn ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Re ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Au ['2000exp_5min']
Running OpenMC for Hf ['2000exp_5min']
Running OpenMC for Cu ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for SS316 ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Al ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Cr ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Ru ['2000exp_5min']
Running OpenMC for Si ['2000exp_5min']
Running OpenMC for Hg ['2000exp_5min']
Running OpenMC for Dy ['2000exp_5min']
Running OpenMC for Fe ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Ce ['2000exp_5min']
Running OpenMC for Cd ['2000exp_5min']
Running OpenMC for Co ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for SS304 ['2000exp_5m

Running OpenMC for Os ['2000exp_5min']
Running OpenMC for Br ['2000exp_5min']
Running OpenMC for Se ['2000exp_5min']
Running OpenMC for Nb ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Mo ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Tm ['2000exp_5min']
Running OpenMC for La ['2000exp_5min']
Running OpenMC for Y ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Ga ['2000exp_5min']
Running OpenMC for Inc600 ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Yb ['2000exp_5min']


Running OpenMC for Ir ['2000exp_5min']
Running OpenMC for Ca ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Pd ['2000exp_5min']
Running OpenMC for Nd ['2000exp_5min']
Running OpenMC for In ['2000exp_5min']
Running OpenMC for Er ['2000exp_5min']
Running OpenMC for Zr ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for Tb ['2000exp_5min']
Running OpenMC for Ti ['2000exp_5min', '1996exp_7hour', '1996exp_5min']
Running OpenMC for I ['2000exp_5min']
Running OpenMC for Te ['2000exp_5min']
Running OpenMC for Cl ['2000exp_5min']


In [10]:
obj = OpenmcActivator(
    activation_data=all_activation_data,
    timestep_units='d',
    chain_file=openmc.config['chain_file'],
)

all_metric_dict = obj.activate(metric_list=['mass', 'decay_heat'])

          294K


[openmc.deplete] t=0.0 s, dt=300.0 s, source=11160000000.0


[openmc.deplete] t=300.0 s, dt=51.00000000000003 s, source=0.0


/home/jon/.neutronicsworkshop_3.12/lib/python3.12/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


[openmc.deplete] t=351.0 s, dt=15.000000000000021 s, source=0.0


...

[openmc.deplete] t=1954.0 s, dt=424.9999999999998 s, source=0.0


[openmc.deplete] t=2379.0 s, dt=604.0000000000003 s, source=0.0


[openmc.deplete] t=2983.0000000000005 s, dt=607.0 s, source=0.0


[openmc.deplete] t=3590.0000000000005 (final operator evaluation)


/home/jon/.neutronicsworkshop_3.12/lib/python3.12/site-packages/openmc/deplete/results.py:94: FutureWarning: The ResultsList.from_hdf5(...) method is no longer necessary and will be removed in a future version of OpenMC. Use Results(...) instead.
  warn(


In [11]:
for entry, (k,exp) in zip(all_metric_dict, element_exp_names):

    openmc_result_dict[k][exp] = entry

Next we process the Fispact simulations results from the IAEA Conderc benchmarks so that they are ready to plot next to the OpenMC simulation results and the experimental benchmark results.

In [12]:
def read_fispact_output(filepath):
    lines = open(filepath).readlines()
    # don't read empty lines
    read = False

    lines = [q for q in lines if q.strip()]
    step = 0
    # get header
    # usually the last # line
    for indx, line in enumerate(lines):
        if read:
            spl = line.strip().split()
            now_step = int(spl[0])
            assert(step +1 == now_step)
            step = now_step
            assert(len(spl) == len(col_names)), print(len(spl), len(col_names), '\n', col_names)
            for indx, val in enumerate(spl):
                key = col_names[indx]
                if key == 'step':
                    d[key].append(int(val))
                else:
                    d[key].append(float(val))
            continue
        if line[0] == '#' and lines[indx+1][0] != '#':
            # this line with the column names
            # terrible
            l = line.strip().split()[1:]
            indx = 0
            new_l = []
            while True:
                if indx >= len(l):
                    break
                if l[indx].isalpha():
                    if l[indx+1][-1] == 'm': # metastable
                        if l[indx+1][:-1].isnumeric():
                            new_l.append(l[indx]+l[indx+1])
                            indx += 2
                        else:
                            new_l.append(l[indx])
                            indx += 1
                    else:
                        if l[indx+1].isnumeric(): # metastable
                            new_l.append(l[indx]+l[indx+1])
                            indx += 2
                        else:
                            new_l.append(l[indx])
                            indx += 1
                else:
                    new_l.append(l[indx])
                    indx += 1
            d = {k:[] for k in new_l}
            read = True
            col_names = copy.deepcopy(new_l)
            continue
    return d


fispact_result_dict = {}
for k,l in experiments.items():
    fispact_result_dict[k] = {}
    for exp in l:
        output_path = here / k / f'TENDL-2017_{exp}.nuclides'
        fispact_result_dict[k][exp] = read_fispact_output(output_path.resolve())

We combine the Fispact results (which are per nuclide) so that we have the total values for decay heat.

In [13]:
fispact_imp_nuclides = {}
for k,l in experiments.items():
    fispact_imp_nuclides[k] = {}
    for exp in l:
        tot = fispact_result_dict[k][exp]['Total']
        indices = [1, len(tot)//2, -1]
        fispact_imp_nuclides[k][exp] = {}
        for i in indices:
            td = {k:v[i] for k,v in fispact_result_dict[k][exp].items() if k not in ['step', 'time', 'uncert', 'Total']}
            td = {k:v for k,v in sorted(td.items(), key=lambda item:item[1], reverse=True)}
            fispact_imp_nuclides[k][exp][i] = td

We now have the OpenMC outputs, Fispact and experimental results in a convenient form  ready for plotting.

The next code block plots the results so that they can be compared.

In [14]:
for k,l in openmc_result_dict.items():
    for exp in l:
        fispact = fispact_result_dict[k][exp] # results in watts per gram
        t_fispact = np.array(fispact['time']) * 365.25 * 60 * 24 # minutes
        fispact_microwatt_gram = np.array(fispact['Total']) * 1e6
        fispact_uncert = np.array(fispact['uncert'])  * 1e6

        # openmc 
        t_openmc = openmc_result_dict[k][exp]['mass']['meta_time_d']
        # t_openmc = np.diff(t_openmc)
        decay_indx = 1
        t0 = t_openmc[decay_indx]
        t_openmc = t_openmc[decay_indx:] - t0
        openmc = openmc_result_dict[k][exp]['decay_heat']['meta_total']
        mass = openmc_result_dict[k][exp]['mass']['meta_total']
        openmc = np.array(openmc) / np.array(mass)
        openmc = openmc[decay_indx:] * 1e6 # watts to microwatts
        # days to minutes
        t_openmc = t_openmc * (60*24)
        # add on 0
        plt.plot(t_openmc, openmc, label='OPENMC', marker='x', alpha=0.5)

        measured = exp_data_dict['data'][k][exp]
        t_measured = exp_data_dict['minutes'][k][exp]
        unc_measured = exp_data_dict['unc'][k][exp]
        # add irradiation time to t_measured
        t_measured = np.array(t_measured)

        for index,sorted_dict in fispact_imp_nuclides[k][exp].items():
            print(index, sorted_dict)

        if 'hour' not in exp:
            t_measured = t_measured / (60*24)
        # plt.errorbar(t_measured, measured, unc_measured, label='Measured', linestyle='--', marker='x')
        plt.fill_between(t_measured, measured-(3*unc_measured), measured+(3*unc_measured),
                         alpha=0.4, label='Measured')
        plt.errorbar(t_fispact, fispact_microwatt_gram, fispact_uncert, label='FISPACT', marker='o', alpha=0.5)
        # plt.plot(t_origen, origen, label='ORIGEN', marker='1', alpha=0.5)
        plt.yscale('log')
        plt.xlabel('Minutes')
        plt.ylabel(r'Specific heat [$\frac{\mu W}{g}$]')
        plt.legend()
        plt.grid()
        plt.title(k + ' (%s)' %(exp))
        plt.savefig(Path('docs') / f'{k}_{exp}.png')
        plt.close()

1 {'N16': 2.93543e-09, 'Tl206': 2.59978e-09, 'Hg205': 1.69815e-09, 'Tl206m': 9.04255e-10, 'Au202': 6.3949e-10, 'Tl202': 2.50339e-10, 'Au200': 7.02249e-11, 'Tl204': 2.96395e-12, 'Au201': 1.01883e-12, 'Au200m': 7.9626e-13, 'Hg203': 3.36274e-13, 'Tl201': 1.8115e-13}
10 {'Tl206': 1.11914e-09, 'Hg205': 8.1943e-10, 'Tl206m': 3.30089e-10, 'Tl202': 2.50285e-10, 'Au200': 6.49417e-11, 'Tl204': 2.96395e-12, 'Au201': 8.80656e-13, 'Au200m': 7.93575e-13, 'Hg203': 3.36255e-13, 'Tl201': 1.80994e-13, 'Au202': 0.0, 'N16': 0.0}
-1 {'Tl202': 2.49809e-10, 'Au200': 3.24796e-11, 'Tl204': 2.9639e-12, 'Hg205': 1.28057e-12, 'Au200m': 7.70167e-13, 'Tl206': 4.92166e-13, 'Hg203': 3.36087e-13, 'Au201': 2.42018e-13, 'Tl201': 1.79611e-13, 'Au202': 0.0, 'Tl206m': 0.0, 'N16': 0.0}


1 {'Rh104': 3.7077e-07, 'Tc100': 4.27197e-08, 'Rh103m': 7.88226e-09, 'Rh104m': 2.5918e-09, 'Rh102m': 4.38424e-11, 'Rh102': 2.92536e-11, 'Tc99m': 7.81155e-12, 'Ru103': 7.38588e-12}
10 {'Rh104': 1.4617e-08, 'Rh103m': 7.44077e-09, 'Rh104m': 1.23002e-09, 'Rh102m': 4.3842e-11, 'Rh102': 2.92535e-11, 'Tc99m': 7.74179e-12, 'Ru103': 7.38546e-12, 'Tc100': 0.0}
-1 {'Rh103m': 5.16968e-09, 'Rh104': 9.98399e-11, 'Rh102m': 4.3839e-11, 'Rh102': 2.92532e-11, 'Rh104m': 1.10789e-11, 'Ru103': 7.38279e-12, 'Tc99m': 7.31525e-12, 'Tc100': 0.0}
1 {'F20': 2.57833e-06, 'N16': 1.06675e-06, 'Ne23': 9.2788e-07, 'Na24': 1.21286e-10, 'Na22': 4.39192e-12}
11 {'Ne23': 3.87633e-09, 'Na24': 1.20828e-10, 'Na22': 4.3919e-12, 'N16': 0.0, 'F20': 0.0}
-1 {'Na24': 1.16275e-10, 'Na22': 4.39179e-12, 'Ne23': 0.0, 'N16': 0.0, 'F20': 0.0}


1 {'Na24': 2.74677e-09, 'Na22': 3.19227e-10}
5 {'Na22': 3.16556e-10, 'Na24': 0.0}
-1 {'Na22': 2.38209e-10, 'Na24': 0.0}


1 {'Ne23': 3.52125e-07, 'F20': 9.81968e-08, 'N16': 6.79188e-09, 'Na24': 1.21205e-10, 'Na22': 4.39191e-12}
10 {'Ne23': 1.53772e-10, 'Na24': 1.20559e-10, 'Na22': 4.3919e-12, 'N16': 0.0, 'F20': 0.0}
-1 {'Na24': 1.1607e-10, 'Na22': 4.39179e-12, 'N16': 0.0, 'F20': 0.0, 'Ne23': 0.0}


1 {'Tl208': 4.48778e-09, 'Tl206m': 9.25759e-10, 'Tl207': 8.99051e-10, 'Pb204m': 7.27546e-10, 'Tl206': 7.16148e-10, 'Pb203m': 2.25806e-10, 'Hg205': 7.86423e-11, 'Pb203': 5.65573e-11, 'Pb209': 8.86775e-12}
10 {'Tl208': 1.28677e-09, 'Pb204m': 6.87594e-10, 'Tl207': 4.04221e-10, 'Tl206': 3.54567e-10, 'Tl206m': 3.35764e-10, 'Pb203': 5.64914e-11, 'Hg205': 3.77752e-11, 'Pb209': 8.69623e-12, 'Pb203m': 0.0}
-1 {'Pb204m': 4.17364e-10, 'Pb203': 5.58832e-11, 'Pb209': 7.31726e-12, 'Tl207': 3.87437e-13, 'Tl206': 2.74051e-13, 'Hg205': 0.0, 'Pb203m': 0.0, 'Tl206m': 0.0, 'Tl208': 0.0}


...

1 {'Y89m': 2.0521e-07, 'Zr89': 7.83876e-08, 'Y90': 1.69535e-08, 'Y92': 2.91698e-09, 'Sr91': 2.22016e-09, 'Y90m': 2.10962e-09, 'Zr95': 1.32127e-09, 'Y91m': 5.85108e-10, 'Y93': 5.63997e-10, 'Nb97': 1.58813e-10, 'Sr87m': 1.56402e-10, 'Zr97': 1.12031e-10, 'Y91': 1.10751e-10, 'Nb97m': 9.18557e-11, 'Sr89': 4.38977e-11, 'Nb95': 1.88267e-11, 'Nb95m': 5.74733e-13, 'Sr90': 8.10292e-15}
5 {'Y89m': 1.52007e-08, 'Zr89': 5.80645e-09, 'Zr95': 1.15698e-09, 'Y90': 7.05487e-10, 'Nb95': 2.66329e-10, 'Y91': 1.02063e-10, 'Sr89': 3.71044e-11, 'Nb95m': 3.50801e-12, 'Sr90': 8.09637e-15, 'Nb97m': 0.0, 'Zr97': 0.0, 'Sr87m': 0.0, 'Nb97': 0.0, 'Y93': 0.0, 'Y91m': 0.0, 'Y90m': 0.0, 'Sr91': 0.0, 'Y92': 0.0}
-1 {'Nb95': 3.3802e-11, 'Zr95': 1.65979e-11, 'Y91': 9.80923e-13, 'Sr89': 1.71977e-13, 'Nb95m': 5.55571e-14, 'Y90': 4.23388e-14, 'Sr90': 7.8898e-15, 'Nb97m': 0.0, 'Zr97': 0.0, 'Sr87m': 0.0, 'Nb97': 0.0, 'Y93': 0.0, 'Y91m': 0.0, 'Y90m': 0.0, 'Sr91': 0.0, 'Y92': 0.0, 'Zr89': 0.0, 'Y89m': 0.0}


1 {'Zr89m': 2.50297e-07, 'Y94': 8.20548e-09, 'Y89m': 6.39044e-09, 'Y90m': 1.15742e-09, 'Y92': 1.13016e-09, 'Y91m': 1.05228e-09, 'Zr89': 1.00025e-09, 'Sr93': 4.12422e-10, 'Sr87m': 1.4779e-10, 'Y90': 1.46386e-10, 'Sr91': 9.42933e-11, 'Y93': 3.18134e-11, 'Y95': 2.51531e-11, 'Zr95': 1.46682e-11, 'Zr97': 4.31997e-12, 'Nb97m': 3.24181e-12, 'Y91': 4.15784e-13, 'Nb97': 1.48419e-13}
10 {'Zr89m': 7.90575e-08, 'Y94': 6.34196e-09, 'Y89m': 2.80199e-09, 'Y90m': 1.12865e-09, 'Y92': 1.10482e-09, 'Zr89': 1.07258e-09, 'Y91m': 9.57178e-10, 'Sr93': 2.15523e-10, 'Y90': 1.48164e-10, 'Sr87m': 1.43636e-10, 'Sr91': 9.35104e-11, 'Y93': 3.25662e-11, 'Y95': 1.57569e-11, 'Zr95': 1.46678e-11, 'Zr97': 4.2993e-12, 'Nb97m': 3.52381e-12, 'Nb97': 5.00111e-13, 'Y91': 4.79801e-13}
-1 {'Y89m': 2.8758e-09, 'Zr89': 1.09852e-09, 'Y94': 1.02823e-09, 'Y90m': 9.44848e-10, 'Y92': 9.4136e-10, 'Y91m': 4.93593e-10, 'Y90': 1.59346e-10, 'Sr87m': 1.17439e-10, 'Sr91': 8.81633e-11, 'Y93': 3.18458e-11, 'Zr89m': 2.3116e-11, 'Zr95': 1.4663e

1 {'Sc50': 5.25902e-08, 'Sc46m': 4.51329e-09, 'Sc48': 4.28504e-09, 'Sc49': 2.16438e-09, 'Ti45': 1.94007e-09, 'Ti51': 4.02114e-10, 'Sc47': 7.49254e-11, 'Sc46': 2.96222e-11, 'Ca47': 5.13477e-12}
11 {'Sc50': 5.20803e-09, 'Sc48': 4.27858e-09, 'Sc49': 2.01992e-09, 'Ti45': 1.89903e-09, 'Ti51': 2.03475e-10, 'Sc47': 7.48649e-11, 'Sc46': 2.97956e-11, 'Ca47': 5.13166e-12, 'Sc46m': 0.0}
-1 {'Sc48': 4.22402e-09, 'Ti45': 1.58307e-09, 'Sc49': 1.12203e-09, 'Sc47': 7.43518e-11, 'Sc46': 2.97872e-11, 'Ca47': 5.10532e-12, 'Ti51': 6.3243e-13, 'Sc46m': 0.0, 'Sc50': 0.0}
1 {'Sc48': 2.8934e-07, 'Sc47': 4.96655e-09, 'Sc46': 2.64665e-09, 'Ti45': 1.85665e-09, 'Ca47': 3.8148e-10, 'Ca45': 5.71372e-11}
5 {'Sc48': 2.75999e-09, 'Sc46': 2.39231e-09, 'Sc47': 4.1818e-10, 'Ca47': 5.90632e-11, 'Ca45': 5.42456e-11, 'Ti45': 0.0}
-1 {'Sc46': 9.491e-11, 'Ca45': 1.03262e-11, 'Ca47': 0.0, 'Ti45': 0.0, 'Sc47': 0.0, 'Sc48': 0.0}


1 {'Sc50': 4.26444e-08, 'Sc48': 4.28445e-09, 'Sc49': 2.15087e-09, 'Ti45': 1.93631e-09, 'Sc46m': 1.42977e-09, 'Ti51': 3.78036e-10, 'Sc47': 7.49199e-11, 'Sc46': 2.97412e-11, 'Ca47': 5.13449e-12}
10 {'Sc48': 4.27662e-09, 'Sc50': 2.57915e-09, 'Sc49': 1.97794e-09, 'Ti45': 1.88672e-09, 'Ti51': 1.65405e-10, 'Sc47': 7.48465e-11, 'Sc46': 2.97953e-11, 'Ca47': 5.13072e-12, 'Sc46m': 0.0}
-1 {'Sc48': 4.22141e-09, 'Ti45': 1.56928e-09, 'Sc49': 1.09075e-09, 'Sc47': 7.43272e-11, 'Sc46': 2.97868e-11, 'Ca47': 5.10406e-12, 'Ti51': 4.78864e-13, 'Sc46m': 0.0, 'Sc50': 0.0}
1 {'N16': 3.61432e-08, 'I128': 1.71788e-08, 'Sb124m': 1.9346e-09, 'I126': 7.04029e-10, 'Te127': 3.63613e-11, 'Sb124n': 2.16576e-11, 'Sb124': 1.02653e-12}
11 {'I128': 1.46328e-08, 'I126': 7.03878e-10, 'Sb124m': 5.14729e-10, 'Te127': 3.61043e-11, 'Sb124n': 1.77593e-11, 'Sb124': 1.16176e-12, 'N16': 0.0}
-1 {'I128': 3.80096e-09, 'I126': 7.02611e-10, 'Sb124m': 7.66799e-11, 'Te127': 3.40153e-11, 'Sb124n': 3.35096e-12, 'Sb124': 1.40027e-12, 'N16'

1 {'N16': 9.97167e-08, 'Te129': 3.60915e-08, 'Sb130m': 3.62218e-09, 'Sb128m': 2.98078e-09, 'Sb126m': 1.78809e-09, 'Te127': 1.60342e-09, 'Sn127m': 6.00892e-10, 'Sb124m': 4.66476e-10, 'Sn125m': 4.57015e-10, 'Sb130': 3.82023e-10, 'Te131': 2.51191e-10, 'Sn127': 7.17795e-11, 'Sb128': 6.8597e-11, 'Te129m': 5.14894e-11, 'Sn123m': 4.51811e-11, 'Sb122m': 3.46628e-11, 'Sb120': 3.00509e-11, 'Te119': 8.90218e-12, 'Te121': 8.47821e-12, 'Te125m': 7.68053e-12, 'Sb129': 7.42897e-12, 'Te127m': 4.2326e-12, 'Te131m': 3.39928e-12, 'Te119m': 3.01396e-12}
11 {'Te129': 3.40865e-08, 'Sb128m': 2.03186e-09, 'Sb130m': 1.92408e-09, 'Te127': 1.59214e-09, 'Sb126m': 1.45667e-09, 'Sb130': 3.45357e-10, 'Sn125m': 3.00817e-10, 'Sn127m': 2.2893e-10, 'Te131': 2.14249e-10, 'Sb124m': 8.8571e-11, 'Sn127': 6.95446e-11, 'Sb128': 6.89098e-11, 'Te129m': 5.14852e-11, 'Sn123m': 4.09024e-11, 'Sb120': 2.3388e-11, 'Sb122m': 1.33897e-11, 'Te119': 8.86541e-12, 'Te121': 8.47711e-12, 'Te125m': 7.68016e-12, 'Sb129': 7.3174e-12, 'Te127m': 